In [ ]:
!pip install rasterio

In [ ]:
import os
import numpy as np
import rasterio

# Path having the raster files
directory = r"E:\Process"

# Function to apply the comparison rule between two arrays
def compare_rasters(arr1, arr2):
    result = np.zeros_like(arr1, dtype=np.uint8)
    result[(arr1 == 1) & (arr2 == 1)] = 1
    result[(arr1 == 1) & (arr2 == 0)] = 2
    result[(arr1 == 0) & (arr2 == 1)] = 3
    return result

# Loop through the years from 1985 to 2022 to compare with the following year
 
for year in range(2014, 2016):
    file1 = os.path.join(directory, f"base{year}.tif")
    file2 = os.path.join(directory, f"base{year+1}.tif")
    output = os.path.join(directory, f"comparacao_{year}_{year+1}.tif")

    print(f"Comparing {file1} with {file2}...")

    with rasterio.open(file1) as src1, rasterio.open(file2) as src2:
        arr1 = src1.read(1)
        arr2 = src2.read(1)
        profile = src1.profile

        result = compare_rasters(arr1, arr2)

        profile.update(dtype=rasterio.uint8, count=1, nodata=255)
        with rasterio.open(saida, 'w', **perfil) as dst:
            dst.write(result, 1)




In [ ]:
!pip install rasterio scipy


In [ ]:
import os
import numpy as np
import rasterio
from scipy.ndimage import label

# Directory containing the comparison files
directory = r"E:\Process"

# Rook connectivity structure (4-neighbors)
structure = np.array([[0, 1, 0],
                      [1, 1, 1],
                      [0, 1, 0]])

# Function to save raster based on a mask
def save_raster_mask(mask, profile, output_path):
    profile.update(dtype=rasterio.uint8, count=1, nodata=0)
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(mask.astype(np.uint8), 1)
# Process all comparison files
for file_name in os.listdir(directory):
    if file_name.startswith("comparacao_") and file_name.endswith(".tif"):
        path = os.path.join(directory, file_name)
        print(f"Processing {file_name}...")

        with rasterio.open(path) as src:
            data = src.read(1)
            profile = src.profile
            # Extract years from the file name
            parts = file_name.replace(".tif", "").split("_")
            year1, year2 = parts[1], parts[2]

            # Separate classes
            for classe, nome in zip([1, 2, 3], ['estavel', 'perda', 'ganho']):
                mask = (data == classe).astype(np.uint8)
                output_path = os.path.join(directory, f"{nome}_{year1}_{year2}.tif")
                save_raster_mask(mask, profile, output_path)
                print(f"  File {nome} saved: {output_path}")

                # Identify connected patches
                binary_mask = mask > 0
                labeled_array, num_features = label(binary_mask, structure=structure)

                # Save labeled raster
                labeled_path = os.path.join(directory, f"{name}_patches_{year1}_{year2}.tif")
                profile.update(dtype=rasterio.int32, nodata=0)
                with rasterio.open(labeled_path, 'w', **profile) as dst:
                    dst.write(labeled_array.astype(np.int32), 1)
                print(f"  Labeled patches saved: {labeled_path}")



In [ ]:
import os
import numpy as np
import rasterio
from scipy.ndimage import binary_dilation
from collections import defaultdict


# Directory containing the patch files
directory = r"E:\Projetos\GeoFM\Patches"

# Rook connectivity structure (4-neighbors)
structure = np.array([[0, 1, 0],
                      [1, 1, 1],
                      [0, 1, 0]])

# Function to load raster
def load_raster(path):
    with rasterio.open(path) as src:
        return src.read(1)

# Function to calculate neighborhood between patches of A and B
def analyze_neighborhood(patches_A, patches_B):
    result = []
    patch_ids_A = np.unique(patches_A)
    patch_ids_A = patch_ids_A[patch_ids_A > 0]  # Ignore background
    for i, patch_id in enumerate(patch_ids_A):
        mask_A = patches_A == patch_id
        dilated = binary_dilation(mask_A, structure=structure)
        border = np.logical_and(dilated, ~mask_A)

        neighbor_values = patches_B[border]
        neighbor_ids = np.unique(neighbor_values[neighbor_values > 0])

        if len(neighbor_ids) == 0:
            viz = 1
        elif len(neighbor_ids) == 1:
            if 0 in neighbor_values:
                viz = 2
            elif np.all(neighbor_values == neighbor_ids[0]):
                viz = 4
            else:
                viz = 2
        else:
            viz = 3

        result.append((int(patch_id), viz))

    return result

# Dictionary to store the results
neighborhood_results = defaultdict(list)

# Process pairs of loss/stable and gain/stable files
for file_name in os.listdir(directory):
    if file_name.startswith("perda_patches_") and file_name.endswith(".tif"):
        parts = file_name.replace(".tif", "").split("_")
        year1, year2 = parts[2], parts[3]
        loss_path = os.path.join(directory, file_name)
        stable_path = os.path.join(directory, f"estavel_patches_{year1}_{year2}.tif")

        patches_A = load_raster(loss_path)
        patches_B = load_raster(stable_path)

        result = analyze_neighborhood(patches_A, patches_B)
        neighborhood_results[f"loss_{year1}_{year2}"] = result

    elif file_name.startswith("ganho_patches_") and file_name.endswith(".tif"):
        parts = file_name.replace(".tif", "").split("_")
        year1, year2 = parts[2], parts[3]
        gain_path = os.path.join(directory, file_name)
        stable_path = os.path.join(directory, f"estavel_patches_{year1}_{year2}.tif")

        patches_A = load_raster(gain_path)
        patches_B = load_raster(stable_path)

        result = analyze_neighborhood(patches_A, patches_B)
        neighborhood_results[f"gain_{year1}_{year2}"] = result

# Example of result visualization
for key, lst in neighborhood_results.items():
    print(f"{key}: {len(lst)} patches analyzed")
    print("Example:", lst[:5])

In [ ]:
import os
import numpy as np
import rasterio
from scipy.ndimage import binary_dilation
from collections import defaultdict

# Directory containing the patch files
directory = r"E:\Projetos\GeoFM\Patches"

# Rook connectivity structure (4-neighbors)
structure = np.array([[0, 1, 0],
                      [1, 1, 1],
                      [0, 1, 0]])

# Function to load raster
def load_raster(path):
    with rasterio.open(path) as src:
        return src.read(1)

# Function to calculate neighborhood between patches of A and B in blocks
def analyze_neighborhood_in_blocks(patches_A, patches_B, block_size=5000):
    result = []
    patch_ids_A = np.unique(patches_A)
    patch_ids_A = patch_ids_A[patch_ids_A > 0]  # Ignore background
    total = len(patch_ids_A)

    for start in range(0, total, block_size):
        end = min(start + block_size, total)
        block_ids = patch_ids_A[start:end]
        print(f"Processing patch blocks {start+1} to {end} of {total}...")

        for patch_id in block_ids:
            mask_A = patches_A == patch_id
            dilated = binary_dilation(mask_A, structure=structure)
            border = np.logical_and(dilated, ~mask_A)

            neighbor_values = patches_B[border]
            neighbor_ids = np.unique(neighbor_values[neighbor_values > 0])

            if len(neighbor_ids) == 0:
                viz = 1
            elif len(neighbor_ids) == 1:
                if 0 in neighbor_values:
                    viz = 2
                elif np.all(neighbor_values == neighbor_ids[0]):
                    viz = 4
                else:
                    viz = 2
            else:
                viz = 3

            result.append((int(patch_id), viz))

    return result   

# Dictionary to store the results
neighborhood_results = defaultdict(list)

# Process pairs of loss/stable and gain/stable files
for file_name in os.listdir(directory):
    if file_name.startswith("perda_patches_") and file_name.endswith(".tif"):
        parts = file_name.replace(".tif", "").split("_")
        year1, year2 = parts[2], parts[3]
        loss_path = os.path.join(directory, file_name)
        stable_path = os.path.join(directory, f"estavel_patches_{year1}_{year2}.tif")

        patches_A = load_raster(loss_path)
        patches_B = load_raster(stable_path)
        result = analyze_neighborhood_in_blocks(patches_A, patches_B)
        neighborhood_results[f"loss_{year1}_{year2}"] = result

    elif file_name.startswith("ganho_patches_") and file_name.endswith(".tif"):
        parts = file_name.replace(".tif", "").split("_")
        year1, year2 = parts[2], parts[3]
        gain_path = os.path.join(directory, file_name)
        stable_path = os.path.join(directory, f"estavel_patches_{year1}_{year2}.tif")

        patches_A = load_raster(gain_path)
        patches_B = load_raster(stable_path)

        result = analyze_neighborhood_in_blocks(patches_A, patches_B)
        neighborhood_results[f"gain_{year1}_{year2}"] = result
# Example of result visualization
for key, lst in neighborhood_results.items():
    print(f"{key}: {len(lst)} patches analyzed")
    print("Example:", lst[:5])

In [ ]:
import os
import numpy as np
import rasterio

directory = r"E:\Process"

print("Converting GeoTIFF files to .npy...")
for file_name in os.listdir(directory):
    if file_name.endswith(".tif") and "_patches_" in file_name:
        tif_path = os.path.join(directory, file_name)
        npy_path = os.path.join(directory, file_name.replace(".tif", ".npy"))
        with rasterio.open(tif_path) as src:
            array = src.read(1)
            np.save(npy_path, array)
        print(f"  Converted: {file_name} -> {os.path.basename(npy_path)}")
print("Conversion completed.\n")

In [ ]:
from scipy.ndimage import binary_dilation
from collections import defaultdict

structure = np.array([[0, 1, 0],
                      [1, 1, 1],
                      [0, 1, 0]])

def analyze_neighborhood_in_blocks(patches_A, patches_B, block_size=5000):
    result = []
    patch_ids_A = np.unique(patches_A)
    patch_ids_A = patch_ids_A[patch_ids_A > 0]
    total = len(patch_ids_A)

    for start in range(0, total, block_size):
        end = min(start + block_size, total)
        block_ids = patch_ids_A[start:end]
        print(f"  Processing patch blocks {start+1} to {end} of {total}...")

        for patch_id in block_ids:
            mask_A = patches_A == patch_id
            dilated = binary_dilation(mask_A, structure=structure)
            border = np.logical_and(dilated, ~mask_A)

            neighbor_values = patches_B[border]
            neighbor_ids = np.unique(neighbor_values[neighbor_values > 0])

            if len(neighbor_ids) == 0:
                viz = 1
            elif len(neighbor_ids) == 1:
                if 0 in neighbor_values:
                    viz = 2
                elif np.all(neighbor_values == neighbor_ids[0]):
                    viz = 4
                else:
                    viz = 2
            else:
                viz = 3

            result.append((int(patch_id), viz))

    return result

# Execute analysis
neighborhood_results = defaultdict(list)

print("Starting neighborhood analysis with .npy files...\n")
for file_name in os.listdir(directory):
    if file_name.startswith("perda_patches_") and file_name.endswith(".npy"):
        parts = file_name.replace(".npy", "").split("_")
        year1, year2 = parts[2], parts[3]
        loss_path = os.path.join(directory, file_name)
        stable_path = os.path.join(directory, f"estavel_patches_{year1}_{year2}.npy")

        if os.path.exists(stable_path):
            patches_A = np.load(loss_path)
            patches_B = np.load(stable_path)
            print(f"Analyzing loss_{year1}_{year2}...")
            result = analyze_neighborhood_in_blocks(patches_A, patches_B)
            neighborhood_results[f"loss_{year1}_{year2}"] = result

    elif file_name.startswith("ganho_patches_") and file_name.endswith(".npy"):
        parts = file_name.replace(".npy", "").split("_")
        year1, year2 = parts[2], parts[3]
        gain_path = os.path.join(directory, file_name)
        stable_path = os.path.join(directory, f"estavel_patches_{year1}_{year2}.npy")

        if os.path.exists(stable_path):
            patches_A = np.load(gain_path)
            patches_B = np.load(stable_path)
            print(f"Analyzing gain_{year1}_{year2}...")
            result = analyze_neighborhood_in_blocks(patches_A, patches_B)
            neighborhood_results[f"gain_{year1}_{year2}"] = result

print("\nNeighborhood analysis completed.")
for key, lst in neighborhood_results.items():
    print(f"{key}: {len(lst)} patches analyzed. Example: {lst[:5]}")

In [ ]:
import os
import numpy as np
import time
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor
from skimage.morphology import binary_dilation



# Directory containing the .npy files
directory = r"E:\Projetos\GeoFM\Patches"

# Rook connectivity structure (4-neighbors)
structure = np.array([[0, 1, 0],
                      [1, 1, 1],
                      [0, 1, 0]])

# Function to process a single patch
def process_patch(patch_id, patches_A, patches_B):
    mask_A = patches_A == patch_id
    dilated = binary_dilation(mask_A, footprint=structure)
    border = np.logical_and(dilated, ~mask_A)

    neighbor_values = patches_B[border]
    neighbor_ids = np.unique(neighbor_values[neighbor_values > 0])

    if len(neighbor_ids) == 0:
        viz = 1
    elif len(neighbor_ids) == 1:
        if 0 in neighbor_values:
            viz = 2
        elif np.all(neighbor_values == neighbor_ids[0]):
            viz = 4
        else:
            viz = 2
    else:
        viz = 3

    return (int(patch_id), viz)

# Function to process blocks of patches with parallelism
def analyze_neighborhood_in_blocks(patches_A, patches_B, block_size=5000):
    result = []
    patch_ids_A = np.unique(patches_A)
    patch_ids_A = patch_ids_A[patch_ids_A > 0]
    total = len(patch_ids_A)

    for start in range(0, total, block_size):
        end = min(start + block_size, total)
        block_ids = patch_ids_A[start:end]
        print(f"  Processing patch blocks {start+1} to {end} of {total}...")
        block_start = time.time()
        with ThreadPoolExecutor() as executor:
            args = ((patch_id, patches_A, patches_B) for patch_id in block_ids)
            block_result = list(executor.map(lambda p: process_patch(*p), args))

        result.extend(block_result)
        print(f"    Time for block: {time.time() - block_start:.2f} seconds")

    return result

# Execute analysis
neighborhood_results = defaultdict(list)

print("Starting neighborhood analysis with .npy files...\n")
for file_name in os.listdir(directory):
    if file_name.startswith("perda_patches_") and file_name.endswith(".npy"):
        parts = file_name.replace(".npy", "").split("_")
        year1, year2 = parts[2], parts[3]
        loss_path = os.path.join(directory, file_name)
        stable_path = os.path.join(directory, f"estavel_patches_{year1}_{year2}.npy")
        if os.path.exists(stable_path):
            patches_A = np.load(loss_path)
            patches_B = np.load(stable_path)
            print(f"Analyzing loss_{year1}_{year2}...")
            result = analyze_neighborhood_in_blocks(patches_A, patches_B)
            neighborhood_results[f"loss_{year1}_{year2}"] = result

    elif file_name.startswith("ganho_patches_") and file_name.endswith(".npy"):
        parts = file_name.replace(".npy", "").split("_")
        year1, year2 = parts[2], parts[3]
        gain_path = os.path.join(directory, file_name)
        stable_path = os.path.join(directory, f"estavel_patches_{year1}_{year2}.npy")

        if os.path.exists(stable_path):
            patches_A = np.load(gain_path)
            patches_B = np.load(stable_path)
            print(f"Analyzing gain_{year1}_{year2}...")
            result = analyze_neighborhood_in_blocks(patches_A, patches_B)
            neighborhood_results[f"gain_{year1}_{year2}"] = result

print("\nNeighborhood analysis completed.")
for key, patch_list in neighborhood_results.items():
    print(f"{key}: {len(patch_list)} patches analyzed. Example: {patch_list[:5]}")

In [ ]:
pip install cupy-cuda12x


In [ ]:
import cupy as cp

print("NNumber of available GPUs:", cp.cuda.runtime.getDeviceCount())
print("GPU name:", cp.cuda.Device(0).name)

In [ ]:
import os
import time
import cupy as cp
import numpy as np
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor
from cupyx.scipy.ndimage import binary_dilation

# Directory containing the .npy files
directory = r"E:\Process"

# Rook connectivity structure (4-neighbors) in CuPy
structure = cp.array([[0, 1, 0],
                      [1, 1, 1],
                      [0, 1, 0]], dtype=cp.uint8)

# Function to process a single patch on the GPU
def process_patch_gpu(patch_id, patches_A_gpu, patches_B_gpu):
    mask_A = patches_A_gpu == patch_id
    dilated = binary_dilation(mask_A, structure=structure)
    border = cp.logical_and(dilated, ~mask_A)

    neighbor_values = patches_B_gpu[border]
    neighbor_ids = cp.unique(neighbor_values[neighbor_values > 0])

    if len(neighbor_ids) == 0:
        viz = 1
    elif len(neighbor_ids) == 1:
        if cp.any(neighbor_values == 0):
            viz = 2
        elif cp.all(neighbor_values == neighbor_ids[0]):
            viz = 4
        else:
            viz = 2
    else:
        viz = 3

    return int(patch_id), viz

# Function to process blocks of patches with parallelism
def analyze_neighborhood_in_blocks_gpu(patches_A, patches_B, block_size=5000):
    result = []
    patch_ids_A = cp.unique(patches_A)
    patch_ids_A = patch_ids_A[patch_ids_A > 0]
    total = len(patch_ids_A)

    for start in range(0, total, block_size):
        end = min(start + block_size, total)
        block_ids = patch_ids_A[start:end]
        print(f"  Processing patch blocks {start+1} to {end} of {total}...")
        block_start = time.time()
        # Transfer block of IDs to CPU for parallelism
        block_ids_cpu = cp.asnumpy(block_ids)
        with ThreadPoolExecutor() as executor:
            args = ((patch_id, patches_A, patches_B) for patch_id in block_ids_cpu)
            block_result = list(executor.map(lambda p: process_patch_gpu(*p), args))

        result.extend(block_result)
        print(f"    Time for block: {time.time() - block_start:.2f} seconds")

    return result

# Execute analysis
neighborhood_results = defaultdict(list)

print("Starting neighborhood analysis with CuPy and GPU...\n")
for file_name in os.listdir(directory):
    if file_name.startswith("perda_patches_") and file_name.endswith(".npy"):
        parts = file_name.replace(".npy", "").split("_")
        year1, year2 = parts[2], parts[3]
        loss_path = os.path.join(directory, file_name)
        stable_path = os.path.join(directory, f"estavel_patches_{year1}_{year2}.npy")

        if os.path.exists(stable_path):
            patches_A = cp.load(loss_path)
            patches_B = cp.load(stable_path)
            print(f"Analyzing loss_{year1}_{year2}...")
            result = analyze_neighborhood_in_blocks_gpu(patches_A, patches_B)
            neighborhood_results[f"loss_{year1}_{year2}"] = result

    elif file_name.startswith("ganho_patches_") and file_name.endswith(".npy"):
        parts = file_name.replace(".npy", "").split("_")
        year1, year2 = parts[2], parts[3]
        gain_path = os.path.join(directory, file_name)
        stable_path = os.path.join(directory, f"estavel_patches_{year1}_{year2}.npy")
        if os.path.exists(stable_path):
            patches_A = cp.load(gain_path)
            patches_B = cp.load(stable_path)
            print(f"Analyzing gain_{year1}_{year2}...")
            result = analyze_neighborhood_in_blocks_gpu(patches_A, patches_B)
            neighborhood_results[f"gain_{year1}_{year2}"] = result

print("\nNeighborhood analysis completed.")
for key, lst in neighborhood_results.items():
    print(f"{key}: {len(lst)} patches analyzed. Example: {lst[:5]}")
